#  Visual English

###  Eryk Wdowiak

This notebook attempts to illustrate the English text that we're using to develop a neural machine translator.

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.collocations import *

# import string
# import re

from wordcloud import WordCloud

In [ ]:
import mxnet as mx
from mxnet import gluon
from mxnet import nd
import gluonnlp as nlp

from data import transform_data_word2vec, preprocess_dataset
from model import SG, CBOW
from utils import print_time

context = mx.cpu()

In [ ]:
##  I thought this function would do far more than just run NLTK tokenizer.
##  We'll leave it in place.  It keeps our options open.
def process_line(line):
    tokens = word_tokenize(line)
    return tokens

In [ ]:
##  read in the lemmatized data
df = pd.read_csv('dataset/train-mparamu_v2-lemmatized.en',header=None)
df.columns = ['en_text']
# df.head()

###  frequencies

In [ ]:
##  flatten data to count words
proc_eng = list(map(process_line, df.en_text))
flat_eng = [item for sublist in proc_eng for item in sublist]
freq_eng = FreqDist(flat_eng)

In [ ]:
freq_eng.most_common(20)

### counts

In [ ]:
# create counts
eng_bar_words = [x[0] for x in freq_eng.most_common(25)]
eng_bar_counts = [x[1] for x in freq_eng.most_common(25)]

# put data into dictionary
eng_dict = dict(zip(eng_bar_words, eng_bar_counts))

In [ ]:
# set the color of our bar graphs
color = cm.viridis_r(np.linspace(.4,.8, 30))

In [ ]:
fig, axs = plt.subplots(figsize=(8,4))

axs.bar(eng_bar_words, eng_bar_counts , color=color)
axs.title.set_text('most common English lemmas')

for ax in fig.axes:
    plt.sca(ax)
    plt.xticks(rotation=45)

plt.tight_layout(pad=0)

plt.savefig('wb-en_lemmas.png')
plt.show()

In [ ]:
# create cloud of Sicilian words by frequency
wordcloud = WordCloud(colormap='Spectral').generate_from_frequencies(eng_dict)

plt.figure(figsize=(10,10), facecolor='k')
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.tight_layout(pad=0)

plt.savefig('wb-en_lemma-cloud.png')
plt.show()

### make wordcloud from embeddings

In [ ]:
##  load datafile (so that we can retrieve vocabulary)
datafile = 'dataset/train-mparamu_v3-lemmatized.en.tsv'

##  CBOW model
model = CBOW
parmfile = './logs/en-cbow-r4-e01.params'
fname_insert = 'cbow'

##  skipgram model
# model = SG
# parmfile = './logs/en-skip-r2-e24.params'
# fname_insert = 'skip'

##  both trained with hyperparameters:
output_dim = 300
batch_size = 128

In [ ]:
##  load the data
data = nlp.data.TSVDataset(datafile)
data, vocab, idx_to_counts = preprocess_dataset( data ) 

##  load the model
embedding = model(token_to_idx=vocab.token_to_idx, output_dim=output_dim,
                  batch_size=batch_size, #num_negatives=num_negatives,
                  negatives_weights=mx.nd.array(idx_to_counts))
embedding.load_parameters(parmfile)

##  get the word vectors
wvecs = embedding.embedding_out.weight.data()

##  count words with at least "min_words" appearances
min_words = 10
num_over_min = len( np.array(idx_to_counts)[ np.array(idx_to_counts)>= min_words ] )

print('vocabulary length:    ' + str(len(vocab)))
print('lemmas over ' + str(min_words) + ' times: ' + str(num_over_min))

In [ ]:
##  pairwise cosine similarity
def cos_sim(wordx, wordy):
    xx = wvecs[vocab.token_to_idx[wordx],]
    yy = wvecs[vocab.token_to_idx[wordy],]
    return nd.dot(xx, yy) / (nd.norm(xx) * nd.norm(yy))

##  full matrix of cosine similarity
def cos_mat( vecs ):
    ##  dot product divided by the norms
    xtx = nd.dot( vecs , vecs.T)
    nmx = nd.sqrt( nd.diag(xtx) ).reshape((-1,1))
    cnm = nd.dot( nmx , nmx.T )
    return xtx / cnm

In [ ]:
##  create "WC Dict" ("word-to-cosine dictionary") for wordcloud
def mk_wcdict(word,k_words):
    
    ##  where to start?  first two tokens are: <BOS> <EOS>
    sv_start = 2
    
    ##  get cosine matrix
    cosmat = cos_mat( wvecs[sv_start:-1,] )
    
    ##  get the row of cosines
    idx_to_lookup = vocab.token_to_idx[word] - sv_start
    row_looked_up = cosmat[idx_to_lookup,]
    
    ##  nearest neighbors by cosine similarity
    knn_cosmat = row_looked_up.argsort()[::-1][1:k_words+1].astype(int).asnumpy()
    
    ##  indexes of nearest neighbors in vocab list
    knn_vocab_idx = list(knn_cosmat + sv_start)
    
    ##  get the words and cosine measures
    knn_vocab_words = [vocab.idx_to_token[idx] for idx in knn_vocab_idx]
    knn_vocab_cosines = [cosmat[idx_to_lookup,idx].asnumpy()[0] for idx in knn_cosmat]
        
    ##  return the dictionary for wordcloud
    return dict(zip(knn_vocab_words,knn_vocab_cosines))

In [ ]:
# create a cloud of 25 words for Don Chisciotti!
knn_wc_dict = mk_wcdict('chisciotti',25)
wordcloud = WordCloud(colormap='Spectral').generate_from_frequencies(knn_wc_dict)

plt.figure(figsize=(10,10), facecolor='k')
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.tight_layout(pad=0)

fname = 'wc-en-' + fname_insert + '_chisciotti.png'
plt.savefig(fname)
plt.show()

In [ ]:
# create a cloud of 25 words for Sanciu Panza!
knn_wc_dict = mk_wcdict('sanciu',25)
wordcloud = WordCloud(colormap='Spectral').generate_from_frequencies(knn_wc_dict)

plt.figure(figsize=(10,10), facecolor='k')
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.tight_layout(pad=0)

fname = 'wc-en-' + fname_insert + '_sanciu.png'
plt.savefig(fname)
plt.show()

### bigrams and trigrams

In [ ]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

In [ ]:
eng_bi_finder = BigramCollocationFinder.from_words(flat_eng)
# eng_bi_finder.apply_freq_filter(5)
eng_bi_scored = eng_bi_finder.score_ngrams(bigram_measures.raw_freq)
eng_bi_scored[:10]

In [ ]:
eng_bi_pmi_finder = BigramCollocationFinder.from_words(flat_eng)
eng_bi_pmi_finder.apply_freq_filter(5)
eng_bi_pmi_scored = eng_bi_pmi_finder.score_ngrams(bigram_measures.pmi)
eng_bi_pmi_scored[0:10]

In [ ]:
eng_tri_finder = TrigramCollocationFinder.from_words(flat_eng)
# eng_tri_finder.apply_freq_filter(5)
eng_tri_scored = eng_tri_finder.score_ngrams(trigram_measures.raw_freq)
eng_tri_scored[:10]

In [ ]:
eng_tri_pmi_finder = TrigramCollocationFinder.from_words(flat_eng)
eng_tri_pmi_finder.apply_freq_filter(5)
eng_tri_pmi_scored = eng_tri_pmi_finder.score_ngrams(trigram_measures.pmi)
eng_tri_pmi_scored[0:10]